# Linear Regression Example

Let's walk through the steps of the official documentation example. Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems (the upcoming Consulting Project).

In [1]:
import findspark

In [2]:
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [6]:
# Load training data
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [9]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# You could also pass in additional parameters for regularization, do the reading 
# in ISLR to fully understand that, after that its just some simple parameter calls.
# Check the documentation with Shift+Tab for more info!

In [10]:
# Fit the model
lrModel = lr.fit(training)

In [11]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.00733507102258,0.831375758434,-0.809530795468,2.44119168688,0.519171379529,1.15345919035,-0.298912411281,-0.51285141862,-0.619712827067,0.695615180432]


Intercept:0.14228558260358093


There is a summary attribute that contains even more info!

In [12]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary

Lots of info, here are a few examples:

In [13]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
|          residuals|
+-------------------+
|-11.011130022096554|
| 0.9236590911176538|
|-4.5957401897776675|
|  -20.4201774575836|
|-10.339160314788181|
|-5.9552091439610555|
|-10.726906349283922|
|  2.122807193191233|
|  4.077122222293811|
|-17.316168071241652|
| -4.593044343959059|
|  6.380476690746936|
| 11.320566035059846|
|-20.721971774534094|
| -2.736692773777401|
| -16.66886934252847|
|  8.242186378876315|
|-1.3723486332690233|
|-0.7060332131264666|
|-1.1591135969994064|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154


## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [14]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [16]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                349|
|   mean|0.24777067865576913|
| stddev| 10.223352147064544|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [19]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                152|
|   mean|  0.277824559663334|
| stddev| 10.565901189752331|
|    min|-26.805483428483072|
|    max| 27.111027963108548|
+-------+-------------------+



In [20]:
unlabeled_data = test_data.select('features')

In [21]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



Now we only train on the train_data

In [22]:
correct_model = lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [23]:
test_results = correct_model.evaluate(test_data)

In [24]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-26.951352549249467|
|-24.362734983470126|
|-20.825554139349045|
|-19.869678925461614|
|-18.945285962274557|
|-15.986343702702497|
|-17.550940287238237|
|-18.591555758569736|
| -15.49854379789564|
| -17.78687558506124|
|-18.361590458439384|
|-15.569451431035558|
|-14.097246918091832|
|-17.273524414021036|
| -15.69924273591395|
| -15.57503757542702|
|-18.620629529185372|
|-13.409010152599555|
|-11.408318336226527|
| -8.702054528301616|
+-------------------+
only showing top 20 rows

RMSE: 10.497312166796439


Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [25]:
predictions = correct_model.transform(unlabeled_data)

In [26]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|0.14586912076639696|
|(10,[0,1,2,3,4,5,...| -2.373472199131598|
|(10,[0,1,2,3,4,5,...| 0.6134768803903734|
|(10,[0,1,2,3,4,5,...|0.08691613584707492|
|(10,[0,1,2,3,4,5,...| 0.6700723962699259|
|(10,[0,1,2,3,4,5,...|-1.5078566541808465|
|(10,[0,1,2,3,4,5,...| 0.8318434536331473|
|(10,[0,1,2,3,4,5,...|  1.899348737258632|
|(10,[0,1,2,3,4,5,...|-0.5871152431258493|
|(10,[0,1,2,3,4,5,...| 2.0061905524379373|
|(10,[0,1,2,3,4,5,...| 2.6380748453908147|
|(10,[0,1,2,3,4,5,...|0.19359370772326118|
|(10,[0,1,2,3,4,5,...|-1.2137336713244564|
|(10,[0,1,2,3,4,5,...|  2.217041439478603|
|(10,[0,1,2,3,4,5,...| 0.8770898261627614|
|(10,[0,1,2,3,4,5,...| 0.8122793224958939|
|(10,[0,1,2,3,4,5,...| 4.8481879674825015|
|(10,[0,1,2,3,4,5,...| 0.2556745462340248|
|(10,[0,1,2,3,4,5,...|-1.0593380448063334|
|(10,[0,1,2,3,4,5,...|-3.7088918745385433|
+----------

Okay, so this data is a bit meaningless, so let's explore this same process with some data that actually makes a little more intuitive sense!